## Cargo la base de tiendas que tengo que analizar

In [1391]:
import pandas as pd
from datetime import datetime

In [1392]:
df = pd.read_csv('data/venta por media hora.csv', encoding='utf-16', header=2)

C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_4364\1539848336.py:1: DtypeWarning: Columns (0,4,13,106,458,459,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/venta por media hora.csv', encoding='utf-16', header=2)


In [1393]:
df = df.copy()

In [1394]:
df = df[1:]

In [1395]:
df = df.rename(columns={
    ' ':'año',
    ' .2':'fecha',
    ' .3':'hora'
})

In [1396]:
df = df.drop(columns=[' .1', 'Punto Operacional'])

In [1397]:
df = df.melt(id_vars=['año', 'fecha', 'hora'], var_name='tiendas', value_name='vct').dropna(subset=['vct'])

In [1398]:
df = df.reset_index().drop(columns=['index'])

In [1399]:
df['vct'] = pd.to_numeric(df['vct'].str.replace('.', '').str.replace(',', '.'), errors='coerce')

In [1400]:
df['mes'] = df['fecha'].str.strip().str.split(' ').str[2].str.strip()

In [1401]:
df['dia'] = df['fecha'].str.strip().str.split(' ').str[0].astype(str)

In [1402]:
df['no'] = df['tiendas'].str.split(' ').str[0].astype(int)

In [1403]:
mes = 'Enero Febrero Marzo Abril Mayo Junio Julio Agosto Septiembre Octubre Noviembre Diciembre'.split(' ')
num_mes = '1 2 3 4 5 6 7 8 9 10 11 12'.split(' ')
mes_orden = dict(zip(mes, num_mes))

In [1404]:
df['mes_numerico'] = df['mes'].map(mes_orden)

In [1405]:
df['fecha_parsed'] = df['mes_numerico'] + '/' + df['dia'] + '/' + df['año'].astype(str)

In [1406]:
df['fecha_final'] = pd.to_datetime(df['fecha_parsed'], format='%m/%d/%Y')

In [1407]:
df['nombre_dia'] = df['fecha_final'].dt.day_name()

In [1408]:
df_domingos = df[df['nombre_dia'] == 'Sunday']

In [1409]:
df_domingos['no'] = df_domingos['tiendas'].str.split(' ').str[0].astype(str)

C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_4364\946036946.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_domingos['no'] = df_domingos['tiendas'].str.split(' ').str[0].astype(str)


## Trabajo sobre la muestra de tiendas a analizar

In [1410]:
pad = pd.read_excel('data/padron_horario_extendido.xlsx')

In [1411]:
pad.columns = pad.columns.str.lower().str.strip()

### Cargo el detalle de Horas Autorizadas para las tiendas

In [1412]:
horas_autorizadas = pd.read_excel('data/padron_horario_extendido.xlsx', sheet_name='horas_dom')

In [1413]:
costo_horas_hombre = pd.read_excel('data/padron_horario_extendido.xlsx', sheet_name='costo_ho')

In [1414]:
horas_autorizadas = horas_autorizadas.dropna(axis=1)

In [1415]:
for col in horas_autorizadas.columns:
    if isinstance(col, datetime):
        horas_autorizadas = horas_autorizadas.rename(columns={
            col:pd.to_datetime(col).month_name().lower().strip()
        })
    else:
        horas_autorizadas = horas_autorizadas.rename(columns={
            col:col.lower().strip()
        })

In [1416]:
for col in costo_horas_hombre.columns:
    if isinstance(col, datetime):
        costo_horas_hombre = costo_horas_hombre.rename(columns={
            col:pd.to_datetime(col).month_name().lower().strip()
        })
    else:
        costo_horas_hombre = costo_horas_hombre.rename(columns={
            col:col.lower().strip()
        })

In [1417]:
costo_horas_hombre = costo_horas_hombre.dropna(axis=1)

In [1418]:
# Creamos columna mes-año
df_domingos['mes'] = df_domingos['fecha_final'].dt.month_name()
df_domingos['mes'] = df_domingos['mes'].astype(str).str.lower()

# Contamos domingos por mes
domingos_por_mes = df_domingos.groupby('mes')['fecha_final'].nunique().reset_index()
domingos_por_mes.columns = ['mes', 'cantidad_domingos']

C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_4364\1875726758.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_domingos['mes'] = df_domingos['fecha_final'].dt.month_name()
C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_4364\1875726758.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_domingos['mes'] = df_domingos['mes'].astype(str).str.lower()


In [1419]:
domingos_por_mes

,mes,cantidad_domingos
0,april,4
1,august,5
2,july,4
3,june,5
4,may,4
5,september,4


In [1420]:
horas_autorizadas = horas_autorizadas.melt(id_vars=['no', 'nombre', 'localidad', 'zona', 'gerente regional', 'domingos', 'horario_anterior'], value_name='horas', var_name='mes')

In [1421]:
horas_autorizadas = pd.merge(right=domingos_por_mes, left=horas_autorizadas, on='mes', how='left')

In [1422]:
horas_autorizadas['horas_por_domingo'] = round(horas_autorizadas['horas'] / horas_autorizadas['cantidad_domingos'],1)

In [1423]:
horas_prom_mayo_abril = horas_autorizadas[horas_autorizadas['mes'].isin(['april', 'may'])].groupby(['no'])['horas'].mean().reset_index().rename(columns={'horas':'horas_promedio_abril_y_mayo'})

In [1424]:
horas_prom_mayo_abril['horas_promedio_abril_y_mayo'] = (horas_prom_mayo_abril['horas_promedio_abril_y_mayo'] / 4).round(2)

In [1425]:
horas_autorizadas = pd.merge(horas_autorizadas, horas_prom_mayo_abril, on='no', how='left')

In [1426]:
horas_autorizadas['crecimiento_horas_domingo'] = (horas_autorizadas['horas_por_domingo'] / horas_autorizadas['horas_promedio_abril_y_mayo'] - 1).round(2)

In [1427]:
horas_autorizadas['horas_adicionales_por_domingo'] = horas_autorizadas['horas_por_domingo'] - horas_autorizadas['horas_promedio_abril_y_mayo']

In [1428]:
# Nos quedamos con la última info de horarios por tienda
horarios_domingos = horas_autorizadas[['no', 'horario_anterior']].drop_duplicates()

In [1429]:
horarios_domingos['horario_apertura'] = pd.to_datetime(horarios_domingos['horario_anterior'].str.split(' ').str[0], format='%H:%M', errors='coerce').dt.time
horarios_domingos['horario_cierre'] = pd.to_datetime(horarios_domingos['horario_anterior'].str.split(' ').str[2], format='%H:%M', errors='coerce').dt.time
horarios_domingos['no'] = horarios_domingos['no'].astype(int)

In [1430]:
horas_autorizadas = pd.merge(horas_autorizadas, horarios_domingos[['no', 'horario_apertura', 'horario_cierre']], on='no', how='left')

In [1431]:
# Extraemos hora de inicio
df_domingos['hora_inicio'] = df_domingos['hora'].str.extract(r'Desde (\d{2}:\d{2})')[0]

# Convertimos a objeto time
df_domingos['hora_inicio'] = pd.to_datetime(df_domingos['hora_inicio'], format='%H:%M').dt.time

C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_4364\2576877.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_domingos['hora_inicio'] = df_domingos['hora'].str.extract(r'Desde (\d{2}:\d{2})')[0]
C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_4364\2576877.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_domingos['hora_inicio'] = pd.to_datetime(df_domingos['hora_inicio'], format='%H:%M').dt.time


In [1432]:
horarios_domingos['no'] = horarios_domingos['no'].astype(int)
df_domingos['no'] = df_domingos['no'].astype(int)

C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_4364\2506752472.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_domingos['no'] = df_domingos['no'].astype(int)


In [1433]:
df_domingos = pd.merge(df_domingos, horarios_domingos[['no', 'horario_apertura', 'horario_cierre']], how='left', on='no').dropna(subset=['horario_apertura'])

In [1434]:
# Filtramos ventas fuera del horario normal (es decir, en horario extendido)
ventas_horario_extendido = df_domingos[(df_domingos['hora_inicio'] < df_domingos['horario_apertura']) | (df_domingos['hora_inicio'] >= df_domingos['horario_cierre'])]

In [1435]:
ventas_horario_extendido = ventas_horario_extendido.groupby(['no', 'mes'])['vct'].sum().reset_index().sort_values(by='no')

In [1436]:
# Confirmo que la seleccion de vct en horario extendido es correcta. Lau tiene otros numeros en su reporte
ventas_horario_extendido[ventas_horario_extendido['no'] == 200]

,no,mes,vct
0,200,april,1053180.04
1,200,august,3249136.54
2,200,july,2185956.55
3,200,june,2406038.41
4,200,may,641432.31
5,200,september,2802756.66


In [1437]:
horas_autorizadas = pd.merge(horas_autorizadas, ventas_horario_extendido, on=['no', 'mes'], how='left')

In [1438]:
horas_autorizadas['vct'] = horas_autorizadas['vct'].fillna(0)

In [1439]:
horas_autorizadas['vst'] = horas_autorizadas['vct'] * (1 - 0.2650)

In [1440]:
margen = pd.read_csv('data/margen_comercial_acum.csv', sep=';')

In [1441]:
margen = margen.pivot_table(values='Importe Ars', columns='RUBRO_CONCAT (grupo) 1', index=['Periodo', 'Tienda'], aggfunc='sum').reset_index()

In [1442]:
margen['VENTA SIN TASA'] = margen['VENTA SIN TASA'].fillna(0)

In [1443]:
margen['MARGEN COMERCIAL'] = pd.to_numeric(margen['MARGEN COMERCIAL'].str.replace(',', '.')).round(2)
margen['VENTA SIN TASA'] = pd.to_numeric(margen['VENTA SIN TASA'].str.replace(',', '.')).round(2)

In [1444]:
margen['ratio_margen'] = (margen['MARGEN COMERCIAL'] / margen['VENTA SIN TASA']).round(2)

In [1445]:
margen = margen.fillna(0)

In [1446]:
margen['mes_numerico'] = margen['Periodo'].astype(str).str.split('2025').str[1]
margen['año'] = '2025'

In [1447]:
margen['fecha_parsed'] = margen['mes_numerico'] + '/01/' + margen['año']

In [1448]:
margen['fecha_final'] = pd.to_datetime(margen['fecha_parsed'], format='%m/%d/%Y')

In [1449]:
margen['mes'] = margen['fecha_final'].dt.month_name().str.lower()

In [1450]:
margen['Tienda'] = margen['Tienda'].astype(int)

In [1451]:
margen = margen.rename(columns={'Tienda':'no'})

In [1452]:
horas_autorizadas = pd.merge(horas_autorizadas, margen[['no', 'mes', 'ratio_margen']], how='left', on=['no', 'mes'])

In [1453]:
horas_autorizadas['margen'] = (horas_autorizadas['vst'] * horas_autorizadas['ratio_margen']).round(2)

In [1454]:
horas_autorizadas['gastos_variables'] = horas_autorizadas['vst'] * - 0.02

In [1455]:
costo_horas_hombre = costo_horas_hombre.melt(id_vars=['no', 'nombre', 'localidad', 'zona', 'gerente regional', 'domingos', 'horario_anterior'], value_name='costo_hora_hombre', var_name='mes')
costo_horas_hombre['costo_hora_hombre'] = costo_horas_hombre['costo_hora_hombre'] * - 1

In [1456]:
horas_autorizadas = pd.merge(horas_autorizadas, costo_horas_hombre[['no', 'mes', 'costo_hora_hombre']], on=['no', 'mes'], how='left')

In [1457]:
horas_autorizadas['horas_adicionales_total_mes'] = horas_autorizadas['horas_adicionales_por_domingo'] * horas_autorizadas['cantidad_domingos']

In [1458]:
horas_autorizadas['gasto_de_personal'] = horas_autorizadas['horas_adicionales_total_mes'] * horas_autorizadas['costo_hora_hombre']

In [1459]:
horas_autorizadas

,no,nombre,localidad,zona,gerente regional,domingos,horario_anterior,mes,horas,cantidad_domingos,...,horario_apertura,horario_cierre,vct,vst,ratio_margen,margen,gastos_variables,costo_hora_hombre,horas_adicionales_total_mes,gasto_de_personal
0,425,Av. Brig. Gral. Juan Manuel de Rosas 2553,GBA,1,Nestor Flores,8:00 a 22:00,9:00 a 20:00,april,107.0,4,...,09:00:00,20:00:00,55524.32,4.081038e+04,0.34,13875.53,-816.207504,-4349.724893,5.20,-22618.569445
1,428,Rosario 4558,GBA,1,Nestor Flores,8:00 a 21:30,8:30 a 20:00,april,96.5,4,...,08:30:00,20:00:00,75922.65,5.580315e+04,0.34,18973.07,-1116.062955,-4485.419200,-0.08,358.833536
2,429,Independencia 4731,GBA,1,Nestor Flores,8:00 a 22:00,9:00 a 20:00,april,61.0,4,...,09:00:00,20:00:00,36447.10,2.678862e+04,0.35,9376.02,-535.772370,-4225.767500,5.56,-23495.267300
3,435,Ruta 26 162,GBA,1,Nestor Flores,8:00 a 21:00,9:00 a 21:00,april,105.0,4,...,09:00:00,21:00:00,450.00,3.307500e+02,0.35,115.76,-6.615000,-4276.323000,17.56,-75092.231880
4,436,Parana 6543,GBA,1,Nestor Flores,8:00 a 22:00,9:00 a 20:00,april,85.5,4,...,09:00:00,20:00:00,6713.00,4.934055e+03,0.33,1628.24,-98.681100,-4460.984412,11.12,-49606.146666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,574,Garcia del Rio 3166,CABA,16,Analia Ortiz,8:00 a 22:00,8:00 a 21:00,september,156.0,4,...,08:00:00,21:00:00,1159366.43,8.521343e+05,0.37,315289.70,-17042.686521,-4285.947770,56.24,-241041.702591
590,741,Av. Triunvirato 4210,CABA,16,Analia Ortiz,8:00 a 22:00,10:30 a 20:00,september,115.5,4,...,10:30:00,20:00:00,2699574.31,1.984187e+06,0.33,654781.75,-39683.742357,-4345.243685,54.08,-234990.778506
591,431,Avenida Presidente Teniente General J.D.Peron ...,Gran Buenos Aires,1,Nestor Flores,8:00 a 22:00,9:00 a 20:00,september,118.0,4,...,09:00:00,20:00:00,2510342.66,1.845102e+06,0.34,627334.63,-36902.037102,-4317.896400,50.00,-215894.820000
592,638,Avenida De Mayo 432,Gran Buenos Aires,1,Nestor Flores,7:00 a 21:00,9:00 a 20:00,september,124.5,4,...,09:00:00,20:00:00,2351091.67,1.728052e+06,0.31,535696.24,-34561.047549,-4259.662380,55.16,-234962.976867


In [1460]:
horas_autorizadas['roc'] = (horas_autorizadas['margen'] + horas_autorizadas['gastos_variables'] + horas_autorizadas['gasto_de_personal']).round(2)

In [1461]:
horas_autorizadas.head(1)

,no,nombre,localidad,zona,gerente regional,domingos,horario_anterior,mes,horas,cantidad_domingos,...,horario_cierre,vct,vst,ratio_margen,margen,gastos_variables,costo_hora_hombre,horas_adicionales_total_mes,gasto_de_personal,roc
0,425,Av. Brig. Gral. Juan Manuel de Rosas 2553,GBA,1,Nestor Flores,8:00 a 22:00,9:00 a 20:00,april,107.0,4,...,20:00:00,55524.32,40810.3752,0.34,13875.53,-816.207504,-4349.724893,5.2,-22618.569445,-9559.25


In [1462]:
horas_autorizadas = horas_autorizadas.rename(columns={
    'mes':'mes_ing',
    'horas':'horas_autorizadas',
    'domingos':'horario_domingo',
})

In [1463]:
dic_meses = {
    'january': 'enero',
    'february': 'febrero',
    'march': 'marzo',
    'april': 'abril',
    'may': 'mayo',
    'june': 'junio',
    'july': 'julio',
    'august': 'agosto',
    'september': 'septiembre',
    'october': 'octubre',
    'november': 'noviembre',
    'december': 'diciembre'
}

In [1464]:
horas_autorizadas['mes_esp'] = horas_autorizadas['mes_ing'].map(dic_meses)

In [1465]:
horas_autorizadas.columns

Index(['no', 'nombre', 'localidad', 'zona', 'gerente regional',
       'horario_domingo', 'horario_anterior', 'mes_ing', 'horas_autorizadas',
       'cantidad_domingos', 'horas_por_domingo', 'horas_promedio_abril_y_mayo',
       'crecimiento_horas_domingo', 'horas_adicionales_por_domingo',
       'horario_apertura', 'horario_cierre', 'vct', 'vst', 'ratio_margen',
       'margen', 'gastos_variables', 'costo_hora_hombre',
       'horas_adicionales_total_mes', 'gasto_de_personal', 'roc', 'mes_esp'],
      dtype='object')

In [1466]:
horas_autorizadas = horas_autorizadas[['no', 'nombre', 'localidad', 'zona', 'gerente regional',  'mes_esp', 'horario_domingo', 'horario_anterior', 'horas_autorizadas', 'cantidad_domingos', 'horas_por_domingo', 'horas_promedio_abril_y_mayo', 'crecimiento_horas_domingo', 'horas_adicionales_por_domingo', 'horas_adicionales_total_mes', 'vct', 'vst', 'ratio_margen','margen', 'gastos_variables', 'costo_hora_hombre', 'gasto_de_personal', 'roc']]

In [1467]:
for col in horas_autorizadas.select_dtypes('number').columns:

    if col == 'ratio_margen' or col == 'crecimiento_horas_domingo':
        horas_autorizadas[col] = horas_autorizadas[col].round(2)
        
    else:
        horas_autorizadas[col] = horas_autorizadas[col].round(1)

In [1468]:
horas_autorizadas = horas_autorizadas.pivot_table(values=['horas_autorizadas', 'cantidad_domingos', 'horas_por_domingo', 'horas_promedio_abril_y_mayo', 'crecimiento_horas_domingo', 'horas_adicionales_por_domingo', 'horas_adicionales_total_mes', 'vct', 'vst', 'ratio_margen','margen', 'gastos_variables', 'costo_hora_hombre', 'gasto_de_personal', 'roc'], columns='mes_esp', index=['no', 'nombre', 'localidad', 'zona', 'gerente regional', 'horario_domingo', 'horario_anterior'], aggfunc='sum')

In [1469]:
orden_columnas_primer_nivel = [
    'horas_autorizadas', 'cantidad_domingos', 'horas_por_domingo',
    'horas_promedio_abril_y_mayo', 'crecimiento_horas_domingo',
    'horas_adicionales_por_domingo', 'horas_adicionales_total_mes',
    'vct', 'vst', 'ratio_margen', 'margen', 'gastos_variables',
    'costo_hora_hombre', 'gasto_de_personal', 'roc'
    ]

horas_autorizadas = horas_autorizadas.reindex(orden_columnas_primer_nivel, level=0, axis=1)

In [1470]:
orden_columnas_segundo_nivel = ['abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre']

horas_autorizadas = horas_autorizadas.reindex(orden_columnas_segundo_nivel, level=1, axis=1)

In [1471]:
horas_autorizadas.reset_index().to_excel('results/analisis_tiendas_horario_extendido_domingos_express.xlsx', index_label='indice', sheet_name='data')